In [1]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import warnings
from add_drive_cost import *


In [9]:
wt = pd.read_csv('kpolyakov.spb.csv')
wt['temperature'] = wt['temperature'].fillna(wt['temperature_predictions'])
rd = pd.read_csv('rides_final.csv')
rd['hour_total'] = pd.to_datetime(rd['start_date']).apply(lambda x: x.hour + x.dayofyear * 24)

In [10]:
wt_to_rd = pd.merge(rd, wt, on='hour_total', how='inner')

In [11]:
wt_to_rd

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0_x,unnamed:_0_x,id,start_date,end_date,start_location,start_district,...,hour_category_afternoon,hour_category_evening,hour_category_morning,hour_category_nan,hour_category_night,season_spring,season_summer,season_nan,temperature_predictions,temperature_interpolated
0,0,0,0,0,0,100340,2023-04-22 09:01:03,2023-04-22 09:24:45,шевченко,юго-восточный,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.786905,13.850245
1,1,1,1,1,1,100341,2023-04-22 09:01:07,2023-04-22 09:29:32,чехова,заречный,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.786905,13.850245
2,2,2,2,2,2,100342,2023-04-22 09:04:27,2023-04-22 09:42:46,буденного,октябрьский,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.786905,13.850245
3,3,3,3,3,3,100343,2023-04-22 09:06:34,2023-04-22 09:38:23,труда,центральный,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.786905,13.850245
4,4,4,4,4,4,100344,2023-04-22 09:09:27,2023-04-22 09:40:07,первомайская,северо-западный,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.786905,13.850245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97367,97367,97367,97367,102250,102250,197707,2023-07-31 23:45:06,2023-08-01 00:06:55,озерная,юго-восточный,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,21.748336,20.620245
97368,97368,97368,97368,102251,102251,197708,2023-07-31 23:46:01,2023-08-01 00:25:35,мичурина,ленинский,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,21.748336,20.620245
97369,97369,97369,97369,102252,102252,197709,2023-07-31 23:48:04,2023-08-01 00:09:21,лермонтова,ленинский,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,21.748336,20.620245
97370,97370,97370,97370,102253,102253,197710,2023-07-31 23:51:41,2023-08-01 00:38:04,рябиновая,северо-западный,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,21.748336,20.620245


In [13]:
wt_to_rd = wt_to_rd[['distance','promo','duration_sec','speed','drive_cost','hour_total','temperature','precipitation_total','wind_gust','cloud_cover_total','sunshine_duration']]

In [14]:
wt_to_rd.corr().to_excel('corr_total.xlsx')

In [ ]:
sns.heatmap(wt_to_rd.corr())